In [87]:
!pip install pandas
!pip install scikit_learn
!pip install xgboost
!nvidia-smi

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Tue May 10 20:56:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.60.02    Driver Version: 510.60.02    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A5000    On   | 00000000:41:00.0 Off |                  Off |
| 30%   25C    P8    17W / 230W |  23287MiB / 24564MiB |      0%      Default |
|                               |                      |                  

In [3]:
import os
import numpy as np 
import pandas as pd
import zipfile
import gc
#
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import *
from tensorflow.keras.layers import *

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import log_loss, roc_auc_score


from sklearn.linear_model import LogisticRegression
import xgboost as xgb
print(xgb.__version__)

2022-05-10 19:22:31.053396: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


1.6.1


# 数据导入

In [4]:
#定义读取测试提交格式
sample_submission = pd.read_csv('sample_submission.csv')

#定义训练测试文件夹
IMG_SIZE = 224
TRAIN_FOLDER = 'train/'
TEST_FOLDER =  'test/'

#定义测试训练，其中.npy是由numpy定义的二进制文件，由np.save()/np.load()读取生成
train_image_list = np.load('cats_and_dogs/train_image_list.npy')
test_image_list = np.load('cats_and_dogs/test_image_list.npy')
target = np.load('cats_and_dogs/target.npy')

print("train:",train_image_list.shape,"test:",test_image_list.shape,"target:",target.shape)


#定义sample_submission.csv中与ids相对应的测试标签
ids = [int(x.split('.')[0]) for x in test_image_list]
print("sampling from test label:",ids[:5])

#已经预制模型以标签
train_EB7_ns = np.load('cats_and_dogs/train_EB7_ns.npy')
test_EB7_ns = np.load('cats_and_dogs/test_EB7_ns.npy')
train_EB4_ns = np.load('cats_and_dogs/train_EB4_ns.npy')
test_EB4_ns = np.load('cats_and_dogs/test_EB4_ns.npy')

train: (25000,) test: (12500,) target: (25000,)
sampling from test label: [2345, 170, 5378, 11242, 3465]


## 硬件配置
- CPU： AMD EPYC 7543 32-Core Processor-15核
- 内存：80GB
- GPU： RTX A5000*1-24GB

## 模型使用说明
- 本模型基于三个主要的预测工具，xgboost/logistic_regression/1-MLP的nn,其中每一个都是由train_EB7_ns和train_EB4_ns分别延伸出来
- train_EB7_ns和train_EB4_ns所推导出来的特征向量个数并不一样，其中train_EB7_ns.shape=(25000, 2560),train_EB4_ns.shape=(25000, 1792)
- 为了更好的成绩，以下的模型均使用了梯度裁减，其裁减方式包括两种：**1.**将处于分布边缘的数据归0和1  **2.**直接保留中间数值，其保留标准为占有大概11000左右的数据
- 应该将EB7和EB4分开训练，不然显存会爆掉，出现Memory allocation error on worker 0: Caching allocator错误
- 最好引入python-gc(garbage collect)，否则会出现显存溢出的错误

## 结果说明
- 在使用单一预测方式时，最好的成绩表现为0.03000左右，nn>xgboost>lr
- 最好的成绩出现在xgboost+lr的组合方式上，经过比例调节，能够保持在0.0298左右，但是无法继续下降
- 任何包括了传统神经网络nn结构的组合都会降低其准确率，包括已经试验的xgboost+nn双组合/xgboost+nn+lr三模型组合
- 在验证集上，借助sklearn.metrics.roc_auc_score，预训练特征向量能够维持在5个9，而借助tensorflow_hub与自己训练组合出来的特征向量大概在3个9，不清楚原始作者使用了哪些优化手段。 借助sklearn.metrics.logloss(和kaggle检测方式一样）,神经网络在单一预测中拥有明显优势。

In [5]:
print(train_EB7_ns.shape)
EB7_X_train, EB7_X_val, EB7_y_train, EB7_y_val = train_test_split(train_EB7_ns, 
                                                                  target, 
                                                                  test_size=0.1, 
                                                                  random_state=42)

(25000, 2560)


# 模型EB7的训练

In [9]:
#调用神经网络
input_tensor = Input(EB7_X_train.shape[1:])
x = input_tensor
# x = BatchNormalization()(x)
# x = LayerNormalization()(x)
x = Dropout(0.5)(x) 
# x = Dense(1024,activation='relu',kernel_regularizer=keras.regularizers.l2(1e-3))(x) #加上一层全连接并不能加大准确率
# x = Dropout(0.2)(x)
x = Dense(1,activation='sigmoid')(x) 
#in Model(first_layer,last_layer)    
model1 = Model(input_tensor, x)

model1.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss = tf.keras.losses.BinaryCrossentropy(),
#               metrics=['binary_accuracy'],
              metrics=['accuracy'])


2022-05-10 19:23:08.015152: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-05-10 19:23:08.017213: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:41:00.0 name: NVIDIA RTX A5000 computeCapability: 8.6
coreClock: 1.695GHz coreCount: 64 deviceMemorySize: 23.69GiB deviceMemoryBandwidth: 715.34GiB/s
2022-05-10 19:23:08.017256: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-05-10 19:23:08.029429: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2022-05-10 19:23:08.029504: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2022-05-10 19:23:08.035060: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcufft.so.

batch_size = 256
print("------model1 training-----------")
model1.fit(EB7_X_train, EB7_y_train,batch_size=batch_size,epochs=10)  

In [11]:
print("---------model1 evaluating-----\n")
nn_pred1 = model1.predict(EB7_X_val)
print(roc_auc_score(EB7_y_val, nn_pred1))
print(log_loss(EB7_y_val, nn_pred1))


# 0.9999718398377975
# 0.008397303281173026

test_preds_nn1 = model1.predict(test_EB7_ns)
print(test_preds_nn1.shape)

---------model1 evaluating-----

0.9999737598488568
0.008180712779016221
(12500, 1)


In [6]:
#调用模型logistic回归
EB7_lr = LogisticRegression(C=0.026, max_iter=10000)
EB7_lr.fit(EB7_X_train, EB7_y_train)
val_preds_EB7_lr = EB7_lr.predict_proba(EB7_X_val)[:,1]

test_preds_EB7_lr = EB7_lr.predict_proba(test_EB7_ns)[:,1]
print(test_preds_EB7_lr.shape)

print(roc_auc_score(EB7_y_val, val_preds_EB7_lr))
print(log_loss(EB7_y_val, val_preds_EB7_lr))

# (12500,)
# 0.999992319955763
# 0.008745738095360568


(12500,)
0.999992319955763
0.008745738095360568


In [7]:
# 定义xgb参数并构造训练样本
xgb_params1 = {
    'eta': 0.05,
    'max_depth': 4,
    'subsample': 0.8,
    'colsample_bytree': 0.6,
    'alpha': 0.01,
    'lambda': 1.00,
    'gamma' : 0.02,
    'max_bin': 256,
    'objective': 'reg:logistic',
    'eval_metric': 'auc',
    'verbosity': 0,
    'tree_method': 'gpu_hist', 
    'predictor': 'gpu_predictor'
}
dtrain1 = xgb.DMatrix(EB7_X_train, EB7_y_train)
dval1 = xgb.DMatrix(EB7_X_val, EB7_y_val)
dtest1 = xgb.DMatrix(test_EB7_ns)

In [8]:
# 模型xgboost训练并预测
model = xgb.train(xgb_params1, dtrain1, num_boost_round=100)
val_preds_xgb_1 = model.predict(dval1)
test_preds_xgb_1 = model.predict(dtest1)
print(test_preds_xgb_1.shape)

#输出指标
#使用非加权的结果
print(roc_auc_score(EB7_y_val, val_preds_xgb_1))
print(log_loss(EB7_y_val, val_preds_xgb_1))
#使用加权的测试
print(roc_auc_score(EB7_y_val, 0.5*val_preds_xgb_1+0.5*val_preds_EB7_lr))
print(log_loss(EB7_y_val, 0.5*val_preds_xgb_1+0.5*val_preds_EB7_lr))

# (12500,)
# 0.999992319955763
# 0.008210439385659993
# 0.9999916799520766
# 0.008179385205861714

(12500,)
0.999992319955763
0.008210439385659993
0.9999916799520766
0.008179385205861714


In [12]:
#清除已经用过的EB7模型，不然会导致显存爆掉，在RTX A5000 * 1-显存:24GB下依然会爆掉
del train_EB7_ns, test_EB7_ns, EB7_X_train, EB7_X_val
gc.collect()

2599

In [13]:
EB4_X_train, EB4_X_val, EB4_y_train, EB4_y_val = train_test_split(train_EB4_ns, 
                                                                  target, 
                                                                  test_size=0.1, 
                                                                  random_state=42)
print(test_EB4_ns.shape)

(12500, 1792)


# 模型EB4的训练

In [17]:
#神经网络模型
input_tensor = Input(EB4_X_train.shape[1:])
x = input_tensor
# x = BatchNormalization()(x)
# x = LayerNormalization()(x)
x = Dropout(0.5)(x) 
# x = Dense(1024,activation='relu',kernel_regularizer=keras.regularizers.l2(1e-3))(x) #加上一层全连接并不能加大准确率
# x = Dropout(0.2)(x)
x = Dense(1,activation='sigmoid')(x) 
#in Model(first_layer,last_layer)    
model2 = Model(input_tensor, x)

model2.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss = tf.keras.losses.BinaryCrossentropy(),
#               metrics=['binary_accuracy'],
              metrics=['accuracy'])

In [18]:
print("------model2-----------")    
model2.fit(EB4_X_train, EB4_y_train, batch_size=batch_size,epochs=10)   

------model2-----------
Epoch 1/10
88/88 [==============================] - 1s 4ms/step - loss: 0.2188 - accuracy: 0.9502
Epoch 2/10
88/88 [==============================] - 0s 2ms/step - loss: 0.0552 - accuracy: 0.9971
Epoch 3/10
88/88 [==============================] - 0s 2ms/step - loss: 0.0338 - accuracy: 0.9972
Epoch 4/10
88/88 [==============================] - 0s 2ms/step - loss: 0.0252 - accuracy: 0.9973
Epoch 5/10
88/88 [==============================] - 0s 2ms/step - loss: 0.0201 - accuracy: 0.9976
Epoch 6/10
88/88 [==============================] - 0s 3ms/step - loss: 0.0176 - accuracy: 0.9976
Epoch 7/10
88/88 [==============================] - 0s 2ms/step - loss: 0.0157 - accuracy: 0.9974
Epoch 8/10
88/88 [==============================] - 0s 2ms/step - loss: 0.0144 - accuracy: 0.9977
Epoch 9/10
88/88 [==============================] - 0s 2ms/step - loss: 0.0139 - accuracy: 0.9973
Epoch 10/10
88/88 [==============================] - 0s 2ms/step - loss: 0.0123 - accuracy: 0.

In [19]:
print("---------model2-----\n")
nn_pred2 = model2.predict(EB4_X_val)
print(nn_pred2.shape)
print(roc_auc_score(EB4_y_val, nn_pred2))
print(log_loss(EB4_y_val, nn_pred2))


test_preds_nn2 = model2.predict(test_EB4_ns)
print(test_preds_nn2.shape)

---------model2-----

(2500, 1)
0.9999628797861876
0.010230590942026356
(12500, 1)


In [14]:
#逻辑回归模型
EB4_lr = LogisticRegression(C=0.019, max_iter=10000)
EB4_lr.fit(EB4_X_train, EB4_y_train)
val_preds_EB4_lr = EB4_lr.predict_proba(EB4_X_val)[:,1]

test_preds_EB4_lr = EB4_lr.predict_proba(test_EB4_ns)[:,1]
print(test_preds_EB4_lr.shape)

print(roc_auc_score(EB4_y_val, val_preds_EB4_lr))
print(log_loss(EB4_y_val, val_preds_EB4_lr))

#(12500,)
# 0.9999948799705086
# 0.009172899469261296
#
# 0.9999577597566962
# 0.011528056949842722
# 0.9999897599410172
# 0.009885062067780902

(12500,)
0.9999948799705086
0.009172899469261296


In [15]:
#xgboost模型参数定义
xgb_params2 = {
    'eta': 0.05,
    'max_depth': 4,
    'subsample': 0.85,
    'colsample_bytree': 0.6,
    'alpha': 0.01,
    'lambda': 1.00,
    'gamma' : 0.02,
    'max_bin': 256,
    'objective': 'reg:logistic',
    'eval_metric': 'auc',
    'verbosity': 0,
    'tree_method': 'gpu_hist', 
    'predictor': 'gpu_predictor'
}


dtrain2 = xgb.DMatrix(EB4_X_train, EB4_y_train)
dval2 = xgb.DMatrix(EB4_X_val, EB4_y_val)
dtest2 = xgb.DMatrix(test_EB4_ns)

In [16]:
#xgboost 参数模型
val_preds_xgb_2 = 0
test_preds_xgb_2 = 0

n_seed = 10

for j in range(n_seed):
    xgb_params2['seed'] = 3*j**2+154
    model = xgb.train(xgb_params2, dtrain2, num_boost_round=100)
    val_preds_xgb_2 += model.predict(dval2)/n_seed
    test_preds_xgb_2 += model.predict(dtest2)/n_seed

#输出指标
#使用非加权的结果
print(roc_auc_score(EB4_y_val, val_preds_xgb_2))
print(log_loss(EB4_y_val, val_preds_xgb_2))
#使用加权的测试
print(roc_auc_score(EB4_y_val, 0.5*val_preds_xgb_2+0.5*val_preds_EB4_lr))
print(log_loss(EB4_y_val, 0.5*val_preds_xgb_2+0.5*val_preds_EB4_lr))

#0.9999577597566962
# 0.011528056949842722
# 0.9999897599410172
# 0.009885062067780902

0.9999577597566962
0.011528056949842722
0.9999897599410172
0.009885062067780902


# 结果生成

In [ ]:
# 查看三种模型的形状以便于组合集成
print(test_preds_xgb_1.shape)
print(test_preds_xgb_2.shape)
print(test_preds_EB7_lr.shape)
print(test_preds_EB4_lr.shape)
# print(test_preds_nn1[0].shape)
print(test_preds_nn1.squeeze(1).shape)
print(test_preds_nn2.squeeze(1).shape)

## 单一模型参数遍历

In [78]:
result = 1
round = 0
for i in [0.1,0.2,0.3,0.4,0.45,0.46,0.47,0.48,0.49,0.5,0.51,0.52,0.53,0.54,0.55,0.56,0.57,0.58,0.59,0.6,0.7,0.8,0.9]:
    if log_loss(EB4_y_val, i*val_preds_xgb_2+(1-i)*val_preds_EB4_lr) < result:
        result = log_loss(EB4_y_val, i*val_preds_xgb_2+(1-i)*val_preds_EB4_lr)
        print(i,result)
    round += 1
    print("round:",round," ",)
        

0.1 0.00925957996293372
round: 1  
round: 2  
round: 3  
round: 4  
round: 5  
round: 6  
round: 7  
round: 8  
round: 9  
round: 10  
round: 11  
round: 12  
round: 13  
round: 14  
round: 15  
round: 16  
round: 17  
round: 18  
round: 19  
round: 20  
round: 21  
round: 22  
round: 23  


In [100]:
sample_submission['id'] = ids
subs = np.clip(test_preds_xgb_1,0.0017,0.9974)

sample_submission['label'] = subs
sample_submission.to_csv('submission_solo5.csv', index=False)
sample_submission.head(5)

,id,label
0,2345,0.995380
1,170,0.003377
2,5378,0.992282
3,11242,0.995918
4,3465,0.003456


## 两种模型组合参数遍历

### xgboost + logistic_regression方式

In [86]:
#(xgb1+lr1)+(xgb2+lr2)
for alpha in [0.8,0.84,0.88,0.92,0.96,1.0,1.02,1.04,1.06,1.08,1.1,1.11,1.12,1.13,1.15,1.17,1.19]:
    sample_submission['id'] = ids
    subs = np.clip(alpha*(0.1*test_preds_xgb_1+0.9*test_preds_EB7_lr)+(1-alpha)*(0.1*test_preds_xgb_2+0.9*test_preds_EB4_lr),
                   0, 
                    1)
    print(subs[subs < 0.0008].shape,subs[subs > 0.9984].shape)            
    print(subs[subs < 0.0010].shape,subs[subs > 0.9982].shape)
    print(subs[subs < 0.0011].shape,subs[subs > 0.9981].shape)
    # print(subs[subs < 0.0012].shape,subs[subs > 0.9980].shape)
    # print(subs[subs < 0.0013].shape,subs[subs > 0.9979].shape)
    # print(subs[subs < 0.0014].shape,subs[subs > 0.9978].shape)
    # print(subs[subs < 0.0015].shape,subs[subs > 0.9977].shape)
    # print(subs[subs < 0.0016].shape,subs[subs > 0.9976].shape)
    # print(subs[subs < 0.0017].shape,subs[subs > 0.9975].shape)
    # print(subs[subs < 0.0018].shape,subs[subs > 0.9974].shape)
    # print(subs[subs < 0.0019].shape,subs[subs > 0.9973].shape)
    
    subs[subs > 0.9974] = 1
    # # subs[subs > 0.9974] = 1
    # subs[subs <  0.0008] = 0
    subs[subs <  0.0018] = 0
    sample_submission['label'] = subs
    sample_submission.to_csv('submission_xgb_lr_new_{}.csv'.format(alpha), index=False)
    sample_submission.head(5)

(1275,) (743,)
(1836,) (1006,)
(2073,) (1126,)
(1295,) (755,)
(1863,) (1020,)
(2102,) (1135,)
(1327,) (778,)
(1891,) (1029,)
(2148,) (1159,)
(1372,) (790,)
(1923,) (1051,)
(2184,) (1173,)
(1410,) (807,)
(1972,) (1075,)
(2226,) (1205,)
(1450,) (830,)
(2018,) (1109,)
(2265,) (1226,)
(1473,) (836,)
(2037,) (1128,)
(2279,) (1236,)
(1488,) (854,)
(2061,) (1137,)
(2308,) (1250,)
(1505,) (873,)
(2080,) (1151,)
(2324,) (1262,)
(1537,) (900,)
(2114,) (1166,)
(2349,) (1278,)
(1563,) (913,)
(2136,) (1180,)
(2372,) (1294,)
(1576,) (916,)
(2154,) (1181,)
(2384,) (1297,)
(1594,) (923,)
(2172,) (1192,)
(2396,) (1308,)
(1620,) (931,)
(2178,) (1198,)
(2404,) (1319,)
(1660,) (952,)
(2197,) (1212,)
(2427,) (1334,)
(1691,) (965,)
(2217,) (1229,)
(2450,) (1350,)
(1722,) (985,)
(2234,) (1246,)
(2468,) (1365,)


### xgboost + 神经网络方式

In [61]:
#(xgb1+nn1)+(xgb2+nn2)
for alpha in [1.02,1.04,1.06,1.08,1.1,1.12,1.14,1.16]:
    # sample_submission['id'] = ids
    
    subs = np.clip(alpha*(0.5*test_preds_xgb_1+0.5*test_preds_nn1.squeeze(1))+(1-alpha)*(0.5*test_preds_xgb_2+0.5*test_preds_nn2.squeeze(1)),
                   0,#0.0018
                   1)#0.9974
    #0.9980/0.9979
    #0.0016/0.0017
    print(subs[subs < 0.0012].shape,subs[subs > 0.9980].shape)
    print(subs[subs < 0.0013].shape,subs[subs > 0.9979].shape)
    print(subs[subs < 0.0014].shape,subs[subs > 0.9978].shape)
    print(subs[subs < 0.0015].shape,subs[subs > 0.9977].shape)
    print(subs[subs < 0.0016].shape,subs[subs > 0.9976].shape)
    print(subs[subs < 0.0017].shape,subs[subs > 0.9975].shape)
    print(subs[subs < 0.0018].shape,subs[subs > 0.9974].shape)
    print(subs[subs < 0.0019].shape,subs[subs > 0.9973].shape)
    # # print(subs.shape)
    # subs[subs > 0.9980] = 1
    # subs[subs <  0.0016] = 0
    
#     subs[subs > 0.9980] = 1
#     subs[subs <  0.0017] = 0
    
#     subs[subs > 0.9979] = 1
#     subs[subs <  0.0016] = 0
    
#     subs[subs > 0.9979] = 1
#     subs[subs <  0.0017] = 0
    sample_submission['label'] = subs
    sample_submission.to_csv('submission_xgb_nn_{}.csv'.format(alpha), index=False)
    # sample_submission.head()
    print("success!")

success!


## 三种模型组合的方式

In [93]:
#(xgb1+lr1)+(xgb2+lr2)
for alpha in [0.5,0.8,1.0,1.06,1.1,1.15,1.17,1.19]:
    sample_submission['id'] = ids
    subs = np.clip(alpha*(0.4*test_preds_xgb_1+0.3*test_preds_EB7_lr+0.4*test_preds_nn1.squeeze(1))+(1-alpha)*(0.4*test_preds_xgb_2+0.3*test_preds_EB4_lr+test_preds_nn2.squeeze(1)),
                   0.0018, 
                    0.9974)
    #测试预测裁减的范围
    # print(subs[subs < 0.0008].shape,subs[subs > 0.9984].shape) 
    # print(subs[subs < 0.0010].shape,subs[subs > 0.9998].shape)
    # print(subs[subs < 0.0010].shape,subs[subs > 0.9989].shape)
    # print(subs[subs < 0.0010].shape,subs[subs > 0.9987].shape)
    # print(subs[subs < 0.0010].shape,subs[subs > 0.9984].shape)
    # print(subs[subs < 0.0010].shape,subs[subs > 0.9983].shape)
    # print(subs[subs < 0.0010].shape,subs[subs > 0.9982].shape)
    # print(subs[subs < 0.0011].shape,subs[subs > 0.9981].shape)
    # print(subs[subs < 0.0012].shape,subs[subs > 0.9980].shape)
    # print(subs[subs < 0.0013].shape,subs[subs > 0.9979].shape)
    # print(subs[subs < 0.0014].shape,subs[subs > 0.9978].shape)
    # print(subs[subs < 0.0015].shape,subs[subs > 0.9977].shape)
    # print(subs[subs < 0.0016].shape,subs[subs > 0.9976].shape)
    # print(subs[subs < 0.0017].shape,subs[subs > 0.9975].shape)
    # print(subs[subs < 0.0018].shape,subs[subs > 0.9974].shape)
    # print(subs[subs < 0.0019].shape,subs[subs > 0.9973].shape)
    
#     subs[subs > 0.9984] = 1
#     # # subs[subs > 0.9974] = 1
#     # subs[subs <  0.0008] = 0
    # subs[subs <  0.0010] = 0
    sample_submission['label'] = subs
    sample_submission.to_csv('submission_xgb_lr_new_{}.csv'.format(alpha), index=False)
    sample_submission.head(5)